In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─── Helpers ────────────────────────────────────────────────────────────────
def load_prices(prices_file: str = "prices.txt", instrument: int = 0) -> np.ndarray:
    """
    Auto‐discover `prices.txt` in the cwd or any parent folder,
    read it as whitespace‐delimited, and return column `instrument`.
    """
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(f"{prices_file} not found")
    if not (0 <= instrument < df.shape[1]):
        raise IndexError(f"instrument must be in [0, {df.shape[1]-1}]")
    return df.iloc[:, instrument].values

def compute_rsi(series: pd.Series, window: int = 14) -> pd.Series:
    """Compute the 0–100 RSI of a pandas Series."""
    delta = series.diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.rolling(window, min_periods=window).mean()
    avg_loss = loss.rolling(window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# ─── Main ─────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # user parameters
    instrument   = 3   # which column
    t0, t1       = 100, 800   # time‐step window
    vol_window   = 20    # lookback for rolling volatility
    rsi_window   = 14    # lookback for RSI

    # load price series
    prices_all = load_prices("prices.txt", instrument)
    prices     = prices_all[t0:t1]
    idx        = np.arange(t0, t1)

    # compute pct‐change volatility
    price_series = pd.Series(prices, index=idx)
    returns      = price_series.pct_change().fillna(0)
    volatility   = returns.rolling(vol_window, min_periods=1).std()

    # compute RSI
    rsi = compute_rsi(price_series, window=rsi_window)

    # ─── Plot 1: Price vs Volatility ─────────────────────────────────────────
    fig, ax1 = plt.subplots(figsize=(10, 4))
    ax1.plot(idx, price_series, color="C0", label="Price")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Price", color="C0")
    ax1.tick_params(axis="y", labelcolor="C0")


    # combine legends
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc="lower left")
    plt.title(f"Instrument {instrument}: Price & Rolling Volatility")
    plt.tight_layout()

    # ─── Plot 2: RSI ─────────────────────────────────────────────────────────
    fig2, ax3 = plt.subplots(figsize=(10, 3))
    ax3.plot(idx, rsi, color="C2", label="RSI")
    ax3.axhline(70, color="r", linestyle="--", label="Overbought (70)")
    ax3.axhline(30, color="g", linestyle="--", label="Oversold (30)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("RSI")
    ax3.set_title(f"Instrument {instrument}: RSI ({rsi_window}-bar)")
    ax3.legend(loc="upper left")
    plt.tight_layout()

    plt.show()


In [ ]:
#RSI looks good lets include that into mdodel
#

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─── Helper to auto-find and load the prices column ─────────────────────────
def load_prices(prices_file: str, instrument: int) -> pd.Series:
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r'\s+', header=None)
            break
    else:
        raise FileNotFoundError(f"{prices_file} not found")
    return df.iloc[:, instrument]

# ─── ATR calculation (uses close as both high & low if you only have closes) ──
def compute_atr(close: pd.Series, window: int = 14) -> pd.Series:
    prev = close.shift(1)
    tr1 = (close - close).abs()           # H–L, here zero since only closes
    tr2 = (close - prev).abs()            # H–PrevClose
    tr3 = (close - prev).abs()            # L–PrevClose, same as tr2
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window, min_periods=1).mean()
    return atr

# ─── Main plotting function ────────────────────────────────────────────────
def plot_price_and_atr(prices_file: str, instrument: int, atr_window: int = 14,
                       t0: int = 0, t1: int = None):
    prices = load_prices(prices_file, instrument)
    if t1 is None:
        t1 = len(prices)
    prices = prices.iloc[t0:t1].reset_index(drop=True)
    
    atr = compute_atr(prices, window=atr_window)

    x = np.arange(t0, t0 + len(prices))

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True,
                                    gridspec_kw={"height_ratios": [3, 1]})

    # Price subplot
    ax1.plot(x, prices, color="black", label="Close Price")
    ax1.set_ylabel("Price")
    ax1.set_title(f"Instrument {instrument}: Price & {atr_window}-Bar ATR")
    ax1.legend(loc="upper left")

    # ATR subplot
    ax2.plot(x, atr, color="C1", label=f"ATR ({atr_window})")
    ax2.set_ylabel("ATR")
    ax2.set_xlabel("Time Step")
    ax2.legend(loc="upper left")

    plt.tight_layout()
    plt.show()


# ─── Example usage ─────────────────────────────────────────────────────────
if __name__ == "__main__":
    plot_price_and_atr("prices.txt", instrument=1, atr_window=7, t0=400, t1=600)


In [ ]:
#Regular algorithm I made using a combination of HMA and Kalman filters and tuned relevant paramters. 
# BASE MODEL HERE: ITERATING OFF THIS.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out


def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)


def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out


# ───────── buffered-HMA logic (trend length vs confirmation) ─────────────
def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
    • If current side has lasted ≥N bars, flip on first opposite bar.
    • Else need X consecutive opposite bars to flip.
    """
    out           = np.empty_like(raw)
    state         = raw[0]
    same_streak   = 1
    opp_counter   = 0
    out[0]        = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter  = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state       = r
                same_streak = 1
                opp_counter = 0
            else:
                same_streak = 0
        out[t] = state
    return out


# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 8,
    N_trend        : int   = 6,
    X_confirm      : int   = 3,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    pct_thresh     : float = 0.001,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.3,
    divider_a      : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) Buffered-HMA signal
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman signal, with neutral band on price change
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int)
    k_dir[1:]  = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement, then “hold-last” persistence
    agree      = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state      = agree.copy()
    for t in range(1, n):
        if state[t] == 0:          # disagreement → keep previous regime
            state[t] = state[t-1]

    # 4) Slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    h   = h_series[T1:T2]
    k   = k_series[T1:T2]
    s   = state[T1:T2]

    # 5) Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shade by contiguous state
    y0, y1 = p.min(), p.max()
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi,
                       facecolor="green" if cur == 1 else "red",
                       alpha=0.22, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1,
               facecolor="green" if cur == 1 else "red",
               alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: persistent regime when models disagree\n"
        f"HMA N={N_trend}, X={X_confirm} | Kalman neutral band ±{pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step"); ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout(); plt.show()


# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  = 19,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 240,
        T2          = 440,
    )


In [ ]:
#Merging two models, I noticd in the cheat on how the HMA and Kalman interact and possibly get the best of thw two models

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(f"{prices_file} not found")
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out = np.full(n, np.nan)
    S = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i-period+1:i+1]).sum() / S
    out[:period-1] = series[:period-1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period//2), max(1, int(np.sqrt(period)))
    return wma(2*wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices: np.ndarray, R=0.05, Q_level=4e-3, Q_trend=1e-5) -> np.ndarray:
    """Two-state (level + trend) causal Kalman filter."""
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0,0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_vs_kalman(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    hma_period     : int   = 4,
    # Kalman params
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.5,   # thin black lines
    divider_alpha  : float = 0.3,
):
    # load prices
    prices = load_prices(prices_file, instrument)
    n = len(prices)
    T2 = min(T2, n)

    # compute indicators
    h_series = hma(prices, hma_period)
    k_series = kalman_trend_smoother(prices, R, Q_level, Q_trend)

    # generate long/short signal: +1 when HMA > Kalman, else -1
    signal = np.where(h_series > k_series, 1, -1)

    # slice for plotting
    x = np.arange(T1, T2)
    p = prices[T1:T2]
    h = h_series[T1:T2]
    k = k_series[T1:T2]
    s = signal[T1:T2]

    # plot
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p, label="Price", color="black")
    ax.plot(x, h, "--", label=f"HMA({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers between each bar
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_alpha, zorder=0)

    # shade regions: green for long, red for short
    run_start, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            color = "green" if cur==1 else "red"
            ax.axvspan(run_start, xi, facecolor=color, alpha=0.2, edgecolor=None, zorder= -1)
            run_start, cur = xi, si
    # final span
    color = "green" if cur==1 else "red"
    ax.axvspan(run_start, x[-1]+1, facecolor=color, alpha=0.2, edgecolor=None, zorder= -1)

    ax.set_title(f"Instr {instrument}: Long (green) when HMA>{'{'}Kalman{'}'} else Short")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_vs_kalman(
        instrument  = 19,
        prices_file = "prices.txt",
        hma_period  = 4,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        T1          = 240,
        T2          = 440,
        divider_w   = 0.5,
        divider_alpha = 0.3,
    )


In [ ]:
# #I noticed that I can eliminate choppyness by possibly runnign a linear reg over the past kalman predictiosn and if they are flat then we should avoid a flip state. 
# new funtion into this which when there is a trend change, it runs a linear regression of N window size on the Kalman curve, and normalizes the gradient. 
# If the gradient is between Grad_pos, grad_neg then it wont flip the signal for that prediction and will default back to the last 

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(f"{prices_file} not found")
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period+1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period-1, n):
        out[i] = (w * series[i-period+1:i+1]).sum() / S
    out[:period-1] = series[:period-1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period//2), max(1, int(np.sqrt(period)))
    return wma(2*wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices: np.ndarray,
                          R=0.05, Q_level=4e-3, Q_trend=1e-5) -> np.ndarray:
    """Two-state (level+trend) causal Kalman filter."""
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0,0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ────────────────────────────  signal filter  ────────────────────────────
def filter_by_kalman_gradient(raw_signal: np.ndarray,
                              k_series: np.ndarray,
                              window: int,
                              grad_pos: float,
                              grad_neg: float) -> np.ndarray:
    """
    Suppress flips in raw_signal when the
    local Kalman slope (normalized) lies between grad_neg and grad_pos.
    - window: look-back window size for regression
    - grad_pos, grad_neg: threshold bounds on normalized slope
    """
    n = len(raw_signal)
    final = raw_signal.copy()
    for t in range(1, n):
        if raw_signal[t] != final[t-1]:
            # potential flip at t
            start = max(0, t-window+1)
            y     = k_series[start:t+1]
            x     = np.arange(len(y))
            # linear regression slope
            slope = np.polyfit(x, y, 1)[0]
            # normalize by current level to get relative slope
            level = np.mean(y) if np.mean(y)!=0 else 1.0
            slope_norm = slope / level
            # if slope in [grad_neg, grad_pos], suppress flip
            if grad_neg <= slope_norm <= grad_pos:
                final[t] = final[t-1]
            else:
                final[t] = raw_signal[t]
        else:
            final[t] = final[t-1]
    return final

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_vs_kalman(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 4,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    # regression filter
    reg_window     : int   = 5,
    grad_pos       : float = 0.001,
    grad_neg       : float = -0.001,
    # window
    T1             : int   = 240,
    T2             : int   = 440,
    divider_w      : float = 0.5,
    divider_alpha  : float = 0.3,
):
    # 1) load price series
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 2) compute HMA and Kalman
    h_series = hma(prices, hma_period)
    k_series = kalman_trend_smoother(prices, R, Q_level, Q_trend)

    # 3) raw long/short signal
    raw_sig = np.where(h_series > k_series, 1, -1)

    # 4) filter flips by Kalman gradient
    sig = filter_by_kalman_gradient(raw_sig, k_series,
                                    reg_window, grad_pos, grad_neg)

    # 5) slice for plotting
    x = np.arange(T1, T2)
    p = prices[T1:T2]
    h = h_series[T1:T2]
    k = k_series[T1:T2]
    s = sig[T1:T2]

    # 6) plot
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p, label="Price", color="black")
    ax.plot(x, h, "--", label=f"HMA({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_alpha, zorder=0)

    # shade regions
    run_start, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            color = "green" if cur==1 else "red"
            ax.axvspan(run_start, xi,
                       facecolor=color, alpha=0.2, edgecolor=None, zorder=-1)
            run_start, cur = xi, si
    color = "green" if cur==1 else "red"
    ax.axvspan(run_start, x[-1]+1,
               facecolor=color, alpha=0.2, edgecolor=None, zorder=-1)

    ax.set_title(
        f"Instr {instrument}: HMA vs Kalman w/ {reg_window}-bar gradient filter\n"
        f"Suppress flips if slope∈[{grad_neg:.4f},{grad_pos:.4f}]"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_vs_kalman(
        instrument   = 19,
        prices_file  = "prices.txt",
        hma_period   = 4,
        R            = 0.075,
        Q_level      = 4e-3,
        Q_trend      = 1e-5,
        reg_window   = 3,
        grad_pos     = 0.001,
        grad_neg     = -0.001,
        T1           = 240,
        T2           = 440,
        divider_w    = 0.5,
        divider_alpha= 0.3,
    )


In [ ]:
#Same algorithm but with an N window before entry into a stock -> Some performance improvements but I we miss out on some gains to be had

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ───────── buffered-HMA logic (trend length vs confirmation) ─────────────
def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
    • If current side has lasted ≥N bars, flip on first opposite bar.
    • Else need X consecutive opposite bars to flip.
    """
    out           = np.empty_like(raw)
    state         = raw[0]
    same_streak   = 1
    opp_counter   = 0
    out[0]        = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter  = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state       = r
                same_streak = 1
                opp_counter = 0
            else:
                same_streak = 0
        out[t] = state
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 8,
    N_trend        : int   = 6,
    X_confirm      : int   = 3,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    pct_thresh     : float = 0.001,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.3,
    divider_a      : float = 0.45,
    # new: hold period
    M_hold         : int   = 2,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) Buffered-HMA
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int)
    k_dir[1:]  = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement + hold-last
    agree      = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state      = agree.copy()
    for t in range(1, n):
        if state[t] == 0:
            state[t] = state[t-1]

    # Build segments of contiguous state
    x        = np.arange(T1, T2)
    p        = prices[T1:T2]
    h_plot   = h_series[T1:T2]
    k_plot   = k_series[T1:T2]
    s        = state[T1:T2]

    segments = []
    cur      = s[0]
    seg_start= x[0]
    for xi, si in zip(x[1:], s[1:]):
        if si != cur:
            segments.append((seg_start, xi, cur))
            seg_start, cur = xi, si
    segments.append((seg_start, x[-1]+1, cur))

    # 5) Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h_plot, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, k_plot, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shade regimes only if segment ≥ M_hold, else leave white
    for start, end, r in segments:
        length = end - start
        if length >= M_hold:
            color = "green" if r==1 else "red"
            ax.axvspan(start, end, facecolor=color, alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: causal {M_hold}-bar hold on regime flips\n"
        f"HMA N={N_trend}, X={X_confirm} | Kalman ±{pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout()
    plt.show()


# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  = 19,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 240,
        T2          = 440,
        divider_w   = 0.3,
        divider_a   = 0.45,
        M_hold      = 3,       # white neutral for first 3 bars
    )


In [ ]:
#Trying a convoluion filter -> I think its delayed too much from convolutions. As of now I cant think of a way to eliminate delay without sacraficing entry and exit points  

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ───────── buffered-HMA logic (trend length vs confirmation) ─────────────
def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
    • If current side has lasted ≥N bars, flip on first opposite bar.
    • Else need X consecutive opposite bars to flip.
    """
    out           = np.empty_like(raw)
    state         = raw[0]
    same_streak   = 1
    opp_counter   = 0
    out[0]        = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter  = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state       = r
                same_streak = 1
                opp_counter = 0
            else:
                same_streak = 0
        out[t] = state
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 8,
    N_trend        : int   = 6,
    X_confirm      : int   = 3,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    pct_thresh     : float = 0.001,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.3,
    divider_a      : float = 0.45,
    # causal blur width
    conv_width     : int   = 5,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) Buffered-HMA signal
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman signal, with neutral band
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int)
    k_dir[1:]  = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement + hold-last persistence
    agree      = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state      = agree.copy()
    for t in range(1, n):
        if state[t] == 0:
            state[t] = state[t-1]

    # 4) Causal blur: remove any run shorter than conv_width
    blurred = np.zeros_like(state)
    for t in range(n):
        # look back conv_width bars (including current)
        start = max(0, t - conv_width + 1)
        window = state[start:t+1]
        s = window.sum()
        # only flip if all conv_width bars agree
        if s ==  conv_width:
            blurred[t] =  1
        elif s == -conv_width:
            blurred[t] = -1
        else:
            # otherwise hold previous blurred state
            blurred[t] = blurred[t-1] if t>0 else state[0]
    state = blurred

    # 5) Slice for plotting
    x = np.arange(T1, T2)
    p = prices[T1:T2]
    h = h_series[T1:T2]
    k = k_series[T1:T2]
    s = state[T1:T2]

    # 6) Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shade by contiguous filtered state
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi,
                       facecolor="green" if cur == 1 else "red",
                       alpha=0.22, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1,
               facecolor="green" if cur == 1 else "red",
               alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: causal {conv_width}-bar blur\n"
        f"HMA N={N_trend}, X={X_confirm} | Kalman neutral band ±{pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step"); ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout(); plt.show()


# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  = 19,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 240,
        T2          = 440,
        divider_w   = 0.3,
        divider_a   = 0.45,
        conv_width  = 4,
    )


In [ ]:
# CHEATING BELOW USED FOR EVALUATION IGNORE ANALYSIS BELOW:

In [ ]:
# That little post-processing pass will remove any spurts of length 1 or 2, and force the chart to stay in whichever regime was in effect just before the tiny flip. You can tune M=3, 4 or whatever feels right.

# Because this uses the future (it needs to know how long the run would have been), it’s technically a look-ahead “cheat,” but it’ll let you test your hypothesis about whether those micro-flips are really dragging your P&L down.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices_file and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period+1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period-1, n):
        out[i] = (w * series[i-period+1:i+1]).sum() / S
    out[:period-1] = series[:period-1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period//2), max(1, int(np.sqrt(period)))
    return wma(2*wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5) -> np.ndarray:
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1,1],[0,1]])
    H   = np.array([[1,0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0,0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + K.flatten()*y
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
      • If current side has lasted ≥N bars, flip on first opposite bar.
      • Else need X consecutive opposite bars before flipping.
    """
    out         = np.empty_like(raw)
    state       = raw[0]
    same_streak = 1
    opp_count   = 0
    out[0]      = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_count   = 0
        else:
            opp_count += 1
            if same_streak >= N or opp_count >= X:
                state       = r
                same_streak = 1
                opp_count   = 0
            else:
                same_streak = 0
        out[t] = state
    return out

def remove_short_runs(state: np.ndarray, M: int) -> np.ndarray:
    """
    Look-ahead “cheat”: any run of identical state values shorter than M
    gets merged into the prior regime (or next if at start).
    """
    out = state.copy()
    n   = len(state)
    i = 0
    while i < n:
        # find end j of this run
        j = i + 1
        while j < n and state[j] == state[i]:
            j += 1
        run_len = j - i
        if run_len < M:
            # fill this tiny run with previous (or next if i==0)
            fill = out[i-1] if i>0 else (state[j] if j<n else state[i])
            out[i:j] = fill
        i = j
    return out

# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree_cheat(
    instrument   : int,
    prices_file  : str   = "prices.txt",
    # HMA params
    hma_period   : int   = 8,
    N_trend      : int   = 6,
    X_confirm    : int   = 3,
    # Kalman params
    R            : float = 0.075,
    Q_level      : float = 4e-3,
    Q_trend      : float = 1e-5,
    pct_thresh   : float = 0.001,
    # minimum run-length to enforce
    M_cheat      : int   = 3,
    # window
    T1           : int   = 340,
    T2           : int   = 800,
    divider_w    : float = 0.3,
    divider_a    : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) HMA + buffered signals
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman + neutral band
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int); k_dir[1:] = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement + hold-last
    agree = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state = agree.copy()
    for t in range(1, n):
        if state[t] == 0:
            state[t] = state[t-1]

    # 4) Cheating look-ahead: remove short runs < M_cheat
    state = remove_short_runs(state, M_cheat)

    # 5) Plot slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    h   = h_series[T1:T2]
    k   = k_series[T1:T2]
    s   = state[T1:T2]

    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA({hma_period})", color="orange", lw=2)
    ax.plot(x, k, "-.", label="Kalman", color="green")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shaded regimes
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi,
                       facecolor="green" if cur==1 else "red",
                       alpha=0.25, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1,
               facecolor="green" if cur==1 else "red",
               alpha=0.25, edgecolor=None)

    ax.set_title(f"Instr {instrument}: cheat-filtered regimes (min run={M_cheat})\n"
                 f"HMA N={N_trend},X={X_confirm} | Kalman ±{pct_thresh*100:.2f}%")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree_cheat(
        instrument   = 19,
        prices_file  = "prices.txt",
        hma_period   = 4,
        N_trend      = 5,
        X_confirm    = 1,
        R            = 0.075,
        Q_level      = 4e-3,
        Q_trend      = 1e-5,
        pct_thresh   = 0.001,
        M_cheat      = 5,
        T1           = 240,
        T2           = 440,
    )


In [ ]:
#Checking if my implmentation of the MODEL in Trader.py reflect the cheating algo emplyed here, a match indicates my All_hands_on_deck objective is correct

In [ ]:
# Single-Instrument, Single-Window CheatTrader Simulation

# Parameters
instrument        = 17       # which instrument (0-based)
t_start, t_end    = 240, 440 # time window [t_start, t_end)

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Hyperparameters
hma_period       = 4
N_trend          = 5
X_confirm        = 1
R, Ql, Qt        = 0.075, 4e-3, 1e-5
pct              = 0.001
M_cheat          = 5
capital_per_inst = 10_000

# Helper functions
def wma(x, n):
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    n = int(n)
    return wma(2*wma(x, max(1, n//2)) - wma(x, n),
               max(1, int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0], 0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        s = F @ s
        P = F @ P @ F.T + Q
        y = x[t] - (H @ s)[0]
        S = (H @ P @ H.T)[0,0] + R
        K = (P @ H.T) / S
        s = s + K.flatten() * y
        P = (np.eye(2) - K @ H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1, len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def remove_short_runs(state, M):
    out = state.copy()
    n = len(state); i = 0
    while i < n:
        j = i+1
        while j < n and state[j] == state[i]:
            j += 1
        if (j - i) < M:
            fill = out[i-1] if i > 0 else (state[j] if j < n else state[i])
            out[i:j] = fill
        i = j
    return out

# Load price data
df = None
for folder in (Path.cwd(), *Path.cwd().parents):
    p = folder / "prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
if df is None:
    raise FileNotFoundError("prices.txt not found")
prices_all = df.values.T
full        = prices_all[instrument]

# Compute signals once
h       = hma(full, hma_period)
dg      = np.zeros_like(h); dg[1:] = (h[1:] - h[:-1]) / h[:-1]
h_raw   = np.where(dg > 0, 1, -1)
h_sig   = buffered(h_raw, N_trend, X_confirm)

k       = kalman(full, R, Ql, Qt)
chg     = np.zeros_like(full); chg[1:] = (full[1:] - full[:-1]) / full[:-1]
k_dir   = np.zeros_like(full, int); k_dir[1:] = np.where(k[1:] > k[:-1], 1, -1)
k_sig   = np.where(np.abs(chg) < pct, 0, k_dir)

agree   = np.where((h_sig == k_sig) & (k_sig != 0), h_sig, 0)
state   = agree.copy()
for i in range(1, len(state)):
    if state[i] == 0:
        state[i] = state[i-1]
state_cheat = remove_short_runs(state, M_cheat)

# Simulate for the window and print
positions = []
for t in range(t_start, t_end):
    sig       = state_cheat[t]
    price_now = full[t]
    if sig != 0 and price_now > 0:
        shares = capital_per_inst // price_now
        pos    = int(sig * shares)
    else:
        shares = 0; pos = 0
    print(f"t={t}, price={price_now:.4f}, sig={sig}, shares={shares}, position={pos}")
    positions.append(pos)

# Plot with shaded position holdings
times    = np.arange(t_start, t_end)
prices_w = full[t_start:t_end]
regimes  = state_cheat[t_start:t_end]

# Build contiguous segments of identical regime
segments = []
current, seg_start = regimes[0], times[0]
for t, r in zip(times[1:], regimes[1:]):
    if r != current:
        segments.append((seg_start, t, current))
        seg_start, current = t, r
segments.append((seg_start, times[-1] + 1, current))

plt.figure(figsize=(12,6))
# Shade background by regime
for start, end, r in segments:
    color = "green" if r>0 else "red"
    plt.axvspan(start, end, facecolor=color, alpha=0.2, edgecolor="none")

# Plot price and indicators
plt.plot(times, prices_w, color="black", label="Raw Price")
plt.plot(times, hma(full, hma_period)[t_start:t_end],
         linestyle="--", color="orange", label=f"HMA({hma_period})")
plt.plot(times, kalman(full, R, Ql, Qt)[t_start:t_end],
         linestyle="-.", color="green", label="Kalman")

plt.xlim(times[0], times[-1])
plt.xlabel("Time Step")
plt.ylabel("Price")
plt.title(f"Instrument {instrument}: cheat-filtered regimes (min run={M_cheat})")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# ────────────────────────────  helper functions  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(f"{prices_file} not found")
    return df.iloc[:, instrument].values

def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period+1, dtype=float)
    out  = np.full(n, np.nan)
    S    = w.sum()
    out[:period-1] = series[:period-1]
    for i in range(period-1, n):
        out[i] = (w * series[i-period+1:i+1]).sum() / S
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period//2), max(1, int(np.sqrt(period)))
    return wma(2*wma(series, half) - wma(series, period), sqrtp)

def kalman(prices: np.ndarray, R: float, Ql: float, Qt: float) -> np.ndarray:
    n = len(prices)
    F = np.array([[1,1],[0,1]], float)
    H = np.array([[1,0]], float)
    Qcov = np.diag([Ql, Qt])
    x = np.array([prices[0], 0.0], float)
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        x = F @ x
        P = F @ P @ F.T + Qcov
        y = prices[t] - (H @ x)[0]
        S = (H @ P @ H.T)[0,0] + R
        K = (P @ H.T) / S
        x = x + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P
        out[t] = x[0]
    return out

def filter_by_kalman_gradient(raw: np.ndarray, kseries: np.ndarray,
                              window: int, grad_pos: float,
                              grad_neg: float) -> np.ndarray:
    n = len(raw)
    out = raw.copy()
    for t in range(1, n):
        if raw[t] != out[t-1]:
            start = max(0, t-window+1)
            y = kseries[start:t+1]
            slope = np.polyfit(np.arange(len(y)), y, 1)[0]
            lvl   = y.mean() or 1.0
            mnorm = slope / lvl
            out[t] = out[t-1] if (grad_neg <= mnorm <= grad_pos) else raw[t]
        else:
            out[t] = out[t-1]
    return out

# ────────────────────────────  reference‐signal generator  ────────────────────────────
def make_signal(prices, hma_period, R, Ql, Qt,
                reg_window, grad_pos, grad_neg):
    h = hma(prices, hma_period)
    k = kalman(prices, R, Ql, Qt)
    raw_sig = np.where(h > k, 1, -1)
    return filter_by_kalman_gradient(raw_sig, k,
                                     reg_window, grad_pos, grad_neg)

# ────────────────────────────  parameter grid  ────────────────────────────
param_grid = {
    'hma_period': [3, 4, 5, 6],
    'R':          [0.05, 0.075],
    'Ql':         [2e-3, 4e-3],
    'Qt':         [1e-5, 2e-5],
    'reg_window':[3, 5, 7],
    'grad_pos':  [0.0005, 0.001, 0.002],
    'grad_neg':  [-0.002, -0.001, -0.0005],
}

# ────────────────────────────  load data & build target  ────────────────────────────
prices = load_prices('prices.txt', instrument=17)
target = make_signal(prices,
                     hma_period=4,
                     R=0.075,
                     Ql=4e-3,
                     Qt=1e-5,
                     reg_window=5,
                     grad_pos=0.001,
                     grad_neg=-0.001)

# ────────────────────────────  grid-search  ────────────────────────────
total = np.prod([len(v) for v in param_grid.values()])
print(f"Starting grid search over {total} parameter combinations...\n")

results = []
for idx, combo in enumerate(product(*param_grid.values()), start=1):
    params = dict(zip(param_grid.keys(), combo))
    sig = make_signal(prices, **params)
    score = np.mean(sig == target)
    results.append({**params, 'match_frac': score})
    print(f"[{idx}/{total}] {params} -> match_frac = {score:.4f}")

# ────────────────────────────  summarize results  ────────────────────────────
df = pd.DataFrame(results)
df_sorted = df.sort_values('match_frac', ascending=False)

print("\nGrid search complete!")
best = df_sorted.iloc[0]
print(f"Best match_frac = {best.match_frac:.4f} with parameters:")
for k in param_grid:
    print(f"  - {k} = {best[k]}")

print("\nTop 10 parameter sets:")
print(df_sorted.head(10).to_string(index=False))


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# ────────────────────────────  shared helpers  ────────────────────────────
def load_prices(prices_file="prices.txt") -> np.ndarray:
    """Search upwards for prices.txt and return (nInst × T) price matrix."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        p = folder / prices_file
        if p.exists():
            df = pd.read_csv(p, sep=r"\s+", header=None)
            return df.values.T
    raise FileNotFoundError(prices_file)

def wma(x: np.ndarray, n: int) -> np.ndarray:
    n = int(n)
    w = np.arange(1, n+1, dtype=float)
    S = w.sum()
    out = np.empty_like(x, dtype=float)
    out[:n-1] = x[:n-1]
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    return out

def hma(x: np.ndarray, n: int) -> np.ndarray:
    half, sq = max(1, n//2), max(1, int(np.sqrt(n)))
    return wma(2*wma(x, half) - wma(x, n), sq)

def kalman(x: np.ndarray, R: float, Ql: float, Qt: float) -> np.ndarray:
    n = len(x)
    F = np.array([[1,1],[0,1]], float)
    H = np.array([[1,0]], float)
    Q = np.diag([Ql,Qt])
    s = np.array([x[0], 0.0], float)
    P = np.eye(2)
    out = np.zeros(n, float)
    for t in range(n):
        s = F @ s
        P = F @ P @ F.T + Q
        y = x[t] - (H @ s)[0]
        S = (H @ P @ H.T)[0,0] + R
        K = (P @ H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K @ H) @ P
        out[t] = s[0]
    return out

def buffered(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    out = np.empty_like(raw, int)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1, len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def remove_short_runs(state: np.ndarray, M: int) -> np.ndarray:
    out = state.copy()
    i, n = 0, len(state)
    while i < n:
        j = i+1
        while j < n and state[j] == state[i]:
            j += 1
        if j - i < M:
            fill = out[i-1] if i>0 else (state[j] if j<n else state[i])
            out[i:j] = fill
        i = j
    return out

# ──────────────────────────  Strategy 1: cheat‐filtered  ──────────────────────────
prices_all = load_prices()
nInst, T = prices_all.shape
print(f"Loaded prices: {nInst} instruments, {T} timesteps")

def getMyPosition(prcSoFar: np.ndarray) -> np.ndarray:
    nInst, t_run = prcSoFar.shape
    positions = np.zeros(nInst, int)
    hma_p, N_tr, X, R, Ql, Qt = 4,5,1,0.075,0.004,1e-5
    pct, M_cheat, cap = 0.001, 5, 10_000
    for i in range(nInst):
        full = prices_all[i]
        h = hma(full, hma_p)
        dg = np.zeros_like(h); dg[1:] = (h[1:]-h[:-1])/h[:-1]
        hr = np.where(dg>0,1,-1)
        hsig = buffered(hr, N_tr, X)
        k = kalman(full, R, Ql, Qt)
        ch = np.zeros_like(full); ch[1:] = (full[1:]-full[:-1])/full[:-1]
        kd = np.zeros_like(full,int); kd[1:] = np.where(k[1:]>k[:-1],1,-1)
        ksig = np.where(np.abs(ch)<pct,0,kd)
        agr = np.where((hsig==ksig)&(ksig!=0),hsig,0)
        state = agr.copy()
        for t in range(1,len(state)):
            if state[t]==0: state[t]=state[t-1]
        st = remove_short_runs(state, M_cheat)
        sig = st[t_run-1]
        price = prcSoFar[i,-1]
        if sig!=0 and price>0:
            nsh = cap//price
            positions[i] = sig*nsh
    return positions

# ──────────────────────────  Strategy 2: grid‐searched  ──────────────────────────
def filter_by_kalman_gradient(raw,kseries,window,grad_pos,grad_neg):
    n=len(raw); out=raw.copy()
    for t in range(1,n):
        if raw[t]!=out[t-1]:
            s = max(0,t-window+1)
            y = kseries[s:t+1]; x_arr = np.arange(len(y))
            slope = np.polyfit(x_arr,y,1)[0]
            lvl = y.mean() or 1.0
            if grad_neg <= slope/lvl <= grad_pos:
                out[t]=out[t-1]
            else:
                out[t]=raw[t]
        else:
            out[t]=out[t-1]
    return out

class GridTrader:
    def __init__(self):
        self.prices_all = prices_all
        nInst, T = prices_all.shape
        self.hma_p, self.R, self.Ql, self.Qt = 4,0.075,0.004,1e-5
        self.reg_w, self.gp, self.gn = 5,0.001,-0.001
        self.capital = 10_000
        self.h_series = np.vstack([hma(prices_all[i],self.hma_p) for i in range(nInst)])
        self.k_series = np.vstack([kalman(prices_all[i],self.R,self.Ql,self.Qt) for i in range(nInst)])
        raw = np.where(self.h_series>self.k_series,1,-1)
        self.signal = np.vstack([filter_by_kalman_gradient(raw[i],self.k_series[i],self.reg_w,self.gp,self.gn)
                                 for i in range(nInst)])
    def Alg(self, prcSoFar):
        nInst,t = prcSoFar.shape; t-=1
        pos = np.zeros(nInst,int)
        price = prcSoFar[:,t]; sig = self.signal[:,t]
        valid = price>0
        nsh = (self.capital//price[valid]).astype(int)
        pos[valid] = sig[valid]*nsh
        return pos

grid = GridTrader()

# ────────────────────────────  Simulation harness  ────────────────────────────
def simulate_pnl(prices, positions, comm=0.0):
    nInst,T = prices.shape
    cash = 0.0; cur = np.zeros(nInst,float); val = 0.0
    totVol = 0.0; pls = []
    for t in range(1,T):
        if t%50==0:
            print(f"P&L sim progress: day {t}/{T-1}")
        pr = prices[:,t]; new = positions[:,t]
        delta = new-cur; dv = np.abs(delta*pr).sum()
        totVol += dv
        cash -= (pr*delta).sum() + dv*comm
        cur = new.copy()
        pv = (cur*pr).sum()
        today = cash+pv - val; val = cash+pv
        pls.append(today)
    a = np.array(pls); mu,sd = a.mean(),a.std(ddof=0)
    sharpe = np.sqrt(249)*mu/sd if sd>0 else 0.0
    ret = val/totVol if totVol>0 else 0.0
    return mu,ret,sd,sharpe,totVol

# build full histories with progress
pos1 = np.zeros((nInst,T),int)
pos2 = np.zeros((nInst,T),int)
print("\nBuilding position histories...")
for t in range(1,T):
    if t%50==0:
        print(f"  at timestep {t}/{T-1}")
    hist = prices_all[:,:t]
    pos1[:,t] = getMyPosition(hist)
    pos2[:,t] = grid.Alg(hist)

# similarity
eq = (pos1==pos2)
print(f"\nOverall match fraction:   {eq.mean():.3%}")
mask = (pos1!=0)|(pos2!=0)
print(f"Match when trading:       {eq[mask].mean():.3%}")

# P&L
print("\nRunning P&L simulations...")
r1 = simulate_pnl(prices_all,pos1)
r2 = simulate_pnl(prices_all,pos2)

df = pd.DataFrame([r1,r2],index=["CheatTrader","GridTrader"],
                  columns=["meanPL","return","stdPL","annSharpe","totVol"])
print("\nPerformance comparison:")
print(df.to_string(float_format="%.4f"))
